<a href="https://colab.research.google.com/github/Sourav7490/ma22c046_2023_PL/blob/main/Assignment-14/Latent_Semantic_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**XVII.1 Natural language processing**

In [30]:
c = {'May31':

    'Two crises convulse a nation: a pandemic and police violence',

    'May30a':
    'Nation’s first astronaut launch to orbit from home soil in nearly adecade',

    'May30b':
    'Death of George Floyd at the hands of police set off protests',

    'May27':
    'SpaceX launch of NASA astronauts is postponed over weather'}

In [31]:
import spacy
from spacy import displacy

# Install dataset: python3 -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

In [32]:
doc0 = nlp('Two crises convulse a nation')

In [33]:
svg = displacy.render(doc0, style="dep", jupyter=False)
with open('sentence0.svg', 'w') as f: f.write(svg)

In [34]:
doc1 = nlp('SpaceX launch of NASA astronauts is postponed over weather')
displacy.render(doc1, style='ent', jupyter=True, options={'distance':90})

In [35]:
[w.lemma_ for w in doc1 if not w.is_stop]

['SpaceX', 'launch', 'NASA', 'astronaut', 'postpone', 'weather']

In [36]:
from spacy.lang.en.stop_words import STOP_WORDS

**XVII.2 Term-document matrix**

In [37]:
import pandas as pd
from scipy.sparse import lil_matrix
d = {}
for j, dok in enumerate(c.keys()):
    tokens = [w.lemma_ for w in nlp(c[dok])
            if not w.is_stop and w.pos_ != 'PUNCT']

for t in tokens:
          d[t] = d.setdefault(t, [])
          d[t] += [j]
A = lil_matrix((len(d.keys()), len(c.keys())), dtype=int)
for i, t in enumerate(d.keys()):
    for j in d[t]:
        A[i, j] = 1
Adf = pd.DataFrame(A.toarray(), index=d.keys(), columns=c.keys()); Adf

,May31,May30a,May30b,May27
SpaceX,0,0,0,1
launch,0,0,0,1
NASA,0,0,0,1
astronaut,0,0,0,1
postpone,0,0,0,1
weather,0,0,0,1


**XVII.3 The idea of LSA**

In [38]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn; seaborn.set();
from numpy.linalg import norm
from scipy.linalg import svd

In [39]:
u, s, vt = svd(A.toarray())

In [40]:
k = 2                               # Limit to rank k
Vt = vt[:k, :]
pd.DataFrame(Vt, columns=c.keys()) # Documents as k-vectors

,May31,May30a,May30b,May27
0,0.0,0.0,0.0,-1.0
1,0.0,0.0,1.0,0.0


In [41]:
US = u[:, :k] @ np.diag(s[:k])
usp = pd.DataFrame(US, index=d.keys()) # Words as k-vectors
usp.head()

,0,1
SpaceX,-1.0,0.0
launch,-1.0,0.0
NASA,-1.0,0.0
astronaut,-1.0,0.0
postpone,-1.0,0.0


In [46]:
astronaut = usp.loc['astronaut', :].to_numpy()

In [ ]:
w = {}; us = np.round(US, 8) # w[(x,y)] = list of words at that point
usr = list(set([tuple(us[i, :]) for i in range(us.shape[0])]))
for i in range(len(usr)):
    w[usr[i]] = []
    for j in range(usp.shape[0]):
        if norm(usp.iloc[j, :] - usr[i]) < 1e-6:
            w[usr[i]] += [usp.index[j]]

fig = plt.figure(figsize=(10, 8)); ax = fig.gca()
ax.arrow(0, 0, crisis[0], crisis[1], width=0.015, alpha=0.3)
ax.arrow(0, 0, police[0], police[1], width=0.015, alpha=0.3)
ax.arrow(0, 0, astronaut[0], astronaut[1], width=0.015, alpha=0.3)
ax.scatter(US[: , 0], US[: ,1], alpha=0.5)
ax.scatter(0, 0, color='r', marker='*', s=150, alpha=0.6);
for i, key in enumerate(w.keys()):
    ax.annotate(', '.join(w[key]), (key[0], key[1]))
ax.set_xlim((-1.5, 0.7)); ax.set_ylim((-0.5, 1.5));
ax.set_title('Alignment of Word Vectors');

In [49]:
c.update(
{
'May30Launch':
'Go NASA! Go SpaceX! Godspeed, Bob and Doug!',
'NYTimes':
'NASA and SpaceX officials more often than not ' +
'just call the pilots of this historic mission Bob and Doug.',
'May30NASAblog':
'The first stage of the SpaceX rocket has landed ' +
'successfully on the droneship, Of Course I Still Love You.',
'May31NYTimes':
'After a 19 hour trip, NASA astronauts Bob and Doug ' +
'successfully docked their capsule and entered the space station.',
})

In [50]:
c.update(
{
'2015Musk':
'West Coast droneship under construction will ' +
'be named Of Course I Still Love You',
'IainBanks':
'These friends of yours are ships. ' +
'Yes, both of them. ' +
'What are they called? ' +
'Of Course I Still Love You and Just Read The Instructions. ' +
'They are not warships? ' +
'With names like that?'
})

In [51]:
from spacy.matcher import PhraseMatcher
terms = ['SpaceX',
        'Of Course I Still Love You',
        'Just Read The Instructions']
patterns = [nlp.make_doc(text) for text in terms]
matcher = PhraseMatcher(nlp.vocab)
matcher.add('TerminologyList', None, *patterns)

In [62]:
from spacy.tokens import Span

def tokensfromdoc(doc):
    d = nlp(doc)
    matches = matcher(d)
    for match_id, start, end in matches:
        term = Span(d, start, end, label='myterms')
        d.ents = list(d.ents) + [term]
    tokens = [w.lemma_ for w in d
              # no pronouns
              if w.pos_ != 'PRON' \
              # no punctuations
              and w.pos_ != 'PUNCT' \
              # not Beginning of a named entity
              and w.ent_iob_ != 'B' \
              # not Inside a named entity
              and w.ent_iob_ != 'I' \
              # not a stop word
              and not w.is_stop]

    tokens += [de.string.rstrip().replace(' ', '_') for de in d.ents]
    return tokens
def dictokens(corpora):
    d = {}
    for j, dok in enumerate(corpora.keys()):
        for t in tokensfromdoc(corpora[dok]):
            d[t] = d.setdefault(t, [])
            d[t] += [j]
    return d

In [63]:
def tdmatrix(d, corpora):
    A = lil_matrix((len(d.keys()), len(corpora.keys())), dtype=int)
    for i, t in enumerate(d.keys()):
        for j in d[t]:
            A[i, j] = 1
    return A

In [ ]:
d = dictokens(c)

In [ ]:
d = dictokens(c)
A = tdmatrix(d, c)
Adf = pd.DataFrame(A.toarray(), index=d.keys(), columns=c.keys())

In [ ]:
Adf.loc[['Of_Course_I_Still_Love_You'], 'NYTimes':].T

**XVII.5 Queries and retrieval**

In [57]:
def retrieve(querytokns, W, Vt, c):

    """Given a list of query word token numbers "querytokns",
    all words vectors "W" and all document vectors "Vt.T"
    extracted from a corpus c, retrieve the documents
    relevant to the query. """
    q = W[querytokns, :].mean(axis=0)
    nrm = norm(q)
    q /= nrm
    idx = np.argsort(Vt.T @ q)[::-1]
    kl = list(c.keys())
    keys = [kl[i] for i in idx]
    docs = [c[k] for k in keys]
    return docs, keys, idx

In [58]:
uu, ss, vvt = svd(A.toarray()) # SVD & rank k approximation
k = 4
U = uu[:, :k]; S = ss[:k];
Vt = vvt[:k, :]               # Document vectors
W = uu[:, :k] @ np.diag(ss[:k]) # Word vectors

In [59]:
myquery = np.where((Adf.index=='astronaut') | (Adf.index=='first'))[0]
docs, keys, idx = retrieve(myquery, W, Vt, c)
docs[:3]

['SpaceX launch of NASA astronauts is postponed over weather',
 'Death of George Floyd at the hands of police set off protests',
 'Nation’s first astronaut launch to orbit from home soil in nearly adecade']

In [60]:
myquery = np.where(Adf.index=='droneship')[0]
docs, keys, idx = retrieve(myquery, W, Vt, c)
docs[:3]

<ipython-input-57-6fb394c42333>:7: RuntimeWarning: Mean of empty slice.
  q = W[querytokns, :].mean(axis=0)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


['SpaceX launch of NASA astronauts is postponed over weather',
 'Death of George Floyd at the hands of police set off protests',
 'Nation’s first astronaut launch to orbit from home soil in nearly adecade']